In [110]:
import pandas as pd
from datetime import datetime

In [111]:
btcusd = pd.read_csv('Demo-output/pre-processed-btcusd.csv', delimiter = "\t")
# Dataset is now stored in a Pandas Dataframe

In [112]:
btcusd.head()

,Unnamed: 0,Dates,Time,open,close,high,low,volume,time
0,0,2013-04-01,00:07:00,93.25,93.30,93.30,93.25,93.300000,00:00
1,1,2013-04-01,00:08:00,100.00,100.00,100.00,100.00,93.300000,00:00
2,2,2013-04-01,00:09:00,93.30,93.30,93.30,93.30,33.676862,00:00
3,3,2013-04-01,00:11:00,93.35,93.47,93.47,93.35,20.000000,00:00
4,4,2013-04-01,00:12:00,93.47,93.47,93.47,93.47,2.021627,00:00


In [113]:
mask = (btcusd['Dates'] >= '2022-07-01') & (btcusd['Dates'] <= '2022-07-30')
selectdata = btcusd.loc[mask]

In [114]:
selectdata = selectdata.drop(['Unnamed: 0'],axis=1)
selectdata = selectdata.reset_index(drop = True)

In [115]:
selectdata.to_csv('Demo-Output/selected_btcusd.csv', sep='\t')

# Change Date and Time format

Since the sentiment analysed from tweet data supposed to have happened before the market price.

Therefore, the sentiment data will be matched the market data an hour before

### btcusd change time and date to 1 hour before for matching sentiment data

In [116]:
selectdata['date'] = selectdata['Dates'] + ' ' + selectdata['Time']

In [117]:
selectdata['date'] = pd.to_datetime(selectdata['date'])

In [118]:
selectdata['date'] = pd.to_numeric(selectdata['date'].values)

In [119]:
for index, row in selectdata.iterrows():
    selectdata.loc[index, 'date'] = row['date'] - 3600000000000

In [120]:
selectdata['date'] = pd.to_numeric(selectdata['date'].values)

In [121]:
selectdata['date'] = pd.to_datetime(selectdata['date'])

In [122]:
selectdata['time'] = selectdata['date']
#pd.to_datetime(selectdata['date'], format='H:00').dt.time
selectdata['date'] = pd.to_datetime(selectdata['date']).dt.date

In [123]:
selectdata["time"] = selectdata["time"].dt.strftime('%H:00')

In [124]:
selectdata

,Dates,Time,open,close,high,low,volume,time,date
0,2022-07-01,00:00:00,19943.000000,19813.000000,19967.000000,19797.177799,20.916900,23:00,2022-06-30
1,2022-07-01,00:01:00,19811.000000,19836.000000,19843.000000,19770.000000,20.380124,23:00,2022-06-30
2,2022-07-01,00:02:00,19830.000000,19757.000000,19853.000000,19757.000000,6.379893,23:00,2022-06-30
3,2022-07-01,00:03:00,19753.702071,19746.000000,19785.000000,19740.469243,7.474126,23:00,2022-06-30
4,2022-07-01,00:04:00,19744.000000,19716.000000,19744.000000,19680.000000,19.455889,23:00,2022-06-30
...,...,...,...,...,...,...,...,...,...
43118,2022-07-30,23:55:00,23647.000000,23628.000000,23649.000000,23612.659134,0.144175,22:00,2022-07-30
43119,2022-07-30,23:56:00,23628.000000,23634.132624,23636.000000,23628.000000,0.243583,22:00,2022-07-30
43120,2022-07-30,23:57:00,23635.000000,23636.000000,23636.000000,23634.132624,0.519763,22:00,2022-07-30
43121,2022-07-30,23:58:00,23644.358096,23636.177719,23644.358096,23636.000000,0.206084,22:00,2022-07-30


In [46]:
#selectdata.to_csv('data.csv', sep='\t')

### Sentiment change date and time format to datetime

In [125]:
sentiment = pd.read_csv('Demo-Output/sentiment.csv', delimiter = "\t")
sentiment = sentiment.drop(['Unnamed: 0'],axis=1)

In [126]:
sentiment["Date"] = pd.to_datetime(sentiment['Date'])

In [127]:
sentiment["Time"] = pd.to_datetime(sentiment['Time'])
sentiment["Time"] = sentiment["Time"].dt.strftime('%H:00')

In [128]:
sentiment

,Date,Time,Final_Sentiment
0,2022-07-01,00:00,0.198706
1,2022-07-01,01:00,0.094379
2,2022-07-01,02:00,0.181538
3,2022-07-01,03:00,0.227096
4,2022-07-01,04:00,0.218856
...,...,...,...
715,2022-07-30,19:00,0.201511
716,2022-07-30,20:00,0.291995
717,2022-07-30,21:00,0.285506
718,2022-07-30,22:00,0.209399


# Matching

Since sentiment data can reflect the hour later crypto price

The data will be matched to the next hour in the market data

And the first 60 rows will not had any sentiment, it will be dropped.

In [84]:
data = selectdata.copy()
for index, row in selectdata.iterrows(): 
    for index2, row2 in sentiment.iterrows():
        if row['date'] == row2['Date']:
            if row['time'] == row2['Time']:
                data.loc[index, 'Sentiment'] = row2['Final_Sentiment']
                break
    print('outside loop:' + str(index))

/var/folders/lm/5jq394gx6yj0y6xk1mlbd31m0000gn/T/ipykernel_90284/4105951522.py:4: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if row['date'] == row2['Date']:


outside loop:0
outside loop:1
outside loop:2
outside loop:3
outside loop:4
outside loop:5
outside loop:6
outside loop:7
outside loop:8
outside loop:9
outside loop:10
outside loop:11
outside loop:12
outside loop:13
outside loop:14
outside loop:15
outside loop:16
outside loop:17
outside loop:18
outside loop:19
outside loop:20
outside loop:21
outside loop:22
outside loop:23
outside loop:24
outside loop:25
outside loop:26
outside loop:27
outside loop:28
outside loop:29
outside loop:30
outside loop:31
outside loop:32
outside loop:33
outside loop:34
outside loop:35
outside loop:36
outside loop:37
outside loop:38
outside loop:39
outside loop:40
outside loop:41
outside loop:42
outside loop:43
outside loop:44
outside loop:45
outside loop:46
outside loop:47
outside loop:48
outside loop:49
outside loop:50
outside loop:51
outside loop:52
outside loop:53
outside loop:54
outside loop:55
outside loop:56
outside loop:57
outside loop:58
outside loop:59
outside loop:60
outside loop:61
outside loop:62
ou

/var/folders/lm/5jq394gx6yj0y6xk1mlbd31m0000gn/T/ipykernel_90284/4105951522.py:4: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if row['date'] == row2['Date']:



outside loop:359
outside loop:360
outside loop:361
outside loop:362
outside loop:363
outside loop:364
outside loop:365
outside loop:366
outside loop:367
outside loop:368
outside loop:369
outside loop:370
outside loop:371
outside loop:372
outside loop:373
outside loop:374
outside loop:375
outside loop:376
outside loop:377
outside loop:378
outside loop:379
outside loop:380
outside loop:381
outside loop:382
outside loop:383
outside loop:384
outside loop:385
outside loop:386
outside loop:387
outside loop:388
outside loop:389
outside loop:390
outside loop:391
outside loop:392
outside loop:393
outside loop:394
outside loop:395
outside loop:396
outside loop:397
outside loop:398
outside loop:399
outside loop:400
outside loop:401
outside loop:402
outside loop:403
outside loop:404
outside loop:405
outside loop:406
outside loop:407
outside loop:408
outside loop:409
outside loop:410
outside loop:411
outside loop:412
outside loop:413
outside loop:414
outside loop:415
outside loop:416
outside loop:

In [88]:
data = data.dropna()

In [89]:
column_names = ["Dates", "Time", "open", "close", "high", "low", "volume", "Sentiment"]
data = data.reindex(columns=column_names)
data.rename(columns = {'Dates':'date'}, inplace = True)
data.rename(columns = {'Time':'time'}, inplace = True)
data.rename(columns = {'Sentiment':'sentiment'}, inplace = True)

In [94]:
data = data.reset_index(drop = True)

In [108]:
data

,date,time,open,close,high,low,volume,sentiment
0,2022-07-01,01:00:00,20275.000000,20289.000000,20308.000000,20273.000000,2.004100,0.198706
1,2022-07-01,01:01:00,20282.000000,20254.000000,20282.000000,20253.000000,3.668172,0.198706
2,2022-07-01,01:02:00,20251.000000,20309.814019,20309.814019,20245.060762,1.532124,0.198706
3,2022-07-01,01:03:00,20313.461178,20319.000000,20371.000000,20313.461178,8.124116,0.198706
4,2022-07-01,01:04:00,20322.000000,20335.000000,20339.000000,20297.000000,2.313103,0.198706
...,...,...,...,...,...,...,...,...
43058,2022-07-30,23:55:00,23647.000000,23628.000000,23649.000000,23612.659134,0.144175,0.209399
43059,2022-07-30,23:56:00,23628.000000,23634.132624,23636.000000,23628.000000,0.243583,0.209399
43060,2022-07-30,23:57:00,23635.000000,23636.000000,23636.000000,23634.132624,0.519763,0.209399
43061,2022-07-30,23:58:00,23644.358096,23636.177719,23644.358096,23636.000000,0.206084,0.209399


### Save the Dataframe to csv

In [109]:
data.to_csv('Demo-Output/merge_data.csv', sep='\t')